<a href="https://colab.research.google.com/github/sidneyjunior0811/Python/blob/main/Project_recomment_anime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [303]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [218]:
animes = pd.read_csv('/content/drive/MyDrive/ANIMES/animes.csv')

In [219]:
animes = animes[['uid', 'title', 'synopsis', 'genre']]

In [220]:
animes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19311 entries, 0 to 19310
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   uid       19311 non-null  int64 
 1   title     19311 non-null  object
 2   synopsis  18336 non-null  object
 3   genre     19311 non-null  object
dtypes: int64(1), object(3)
memory usage: 603.6+ KB


In [221]:
animes = animes.dropna()
animes = animes.drop_duplicates()

In [222]:
animes.head(2)

,uid,title,synopsis,genre
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun..."
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun..."


In [223]:
# Transformar a coluna Genre em Lista
animes['genre'][102]

"['Slice of Life', 'Comedy']"

In [224]:
animes['genre'] = animes['genre'].apply(lambda x: x.replace(" ",""))

In [225]:
animes['genre'] = animes['genre'].apply(lambda x: x.strip("[']")).apply(lambda x: x.split())

In [228]:
animes['genre'][102]

["SliceofLife','Comedy"]

In [231]:
# Removendo \r\n \r\n
animes['synopsis'][3]

'"In order for something to be obtained, something of equal value must be lost." \r\n \r\nAlchemy is bound by this Law of Equivalent Exchange—something the young brothers Edward and Alphonse Elric only realize after attempting human transmutation: the one forbidden act of alchemy. They pay a terrible price for their transgression—Edward loses his left leg, Alphonse his physical body. It is only by the desperate sacrifice of Edward\'s right arm that he is able to affix Alphonse\'s soul to a suit of armor. Devastated and alone, it is the hope that they would both eventually return to their original bodies that gives Edward the inspiration to obtain metal limbs called "automail" and become a state alchemist, the Fullmetal Alchemist. \r\n \r\nThree years of searching later, the brothers seek the Philosopher\'s Stone, a mythical relic that allows an alchemist to overcome the Law of Equivalent Exchange. Even with military allies Colonel Roy Mustang, Lieutenant Riza Hawkeye, and Lieutenant Co

In [272]:
animes['synopsis'] = animes['synopsis'].apply(lambda x: [i.strip("\r\n\\'") for i in x]).apply(lambda x: "".join(x)).apply(lambda x: x.split())

In [274]:
animes.head(1)

,uid,title,synopsis,genre
0,28891,Haikyuu!! Second Season,"[Following, their, participation, at, the, Int...","[Comedy','Sports','Drama','School','Shounen]"


In [276]:
animes['tags'] = animes['synopsis'] + animes['genre']

In [281]:
animes.head(1)

,uid,title,synopsis,genre,tags
0,28891,Haikyuu!! Second Season,"[Following, their, participation, at, the, Int...","[Comedy','Sports','Drama','School','Shounen]","[Following, their, participation, at, the, Int..."


In [282]:
new_animes = animes[['uid','title','tags']]

In [284]:
new_animes.head(1)

,uid,title,tags
0,28891,Haikyuu!! Second Season,"[Following, their, participation, at, the, Int..."


In [287]:
new_animes['tags'] = new_animes['tags'].apply(lambda x: " ".join(x))

<ipython-input-287-5487b24e9c1b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_animes['tags'] = new_animes['tags'].apply(lambda x: " ".join(x))


"Music accompanies the path of the human metronome, the prodigious pianist Kousei Arima. But after the passing of his mother, Saki Arima, Kousei falls into a downward spiral, rendering him unable to hear the sound of his own piano. Two years later, Kousei still avoids the piano, leaving behind his admirers and rivals, and lives a colorless life alongside his friends Tsubaki Sawabe and Ryouta Watari. However, everything changes when he meets a beautiful violinist, Kaori Miyazono, who stirs up his world and sets him on a journey to face music again. Based on the manga series of the same name, Shigatsu wa Kimi no Uso approaches the story of Kouseis recovery as he discovers that music is more than playing each note perfectly, and a single melody can bring in the fresh spring air of April. [Written by MAL Rewrite] Drama','Music','Romance','School','Shounen"

In [291]:
new_animes['uid'].count()

15453

In [301]:
cv = CountVectorizer(max_features = 5000, stop_words = 'english')
vectors = cv.fit_transform(new_animes['tags']).toarray()

In [304]:
similarity = cosine_similarity(vectors)

In [306]:
sorted(list(enumerate(similarity[0])), reverse= True, key= lambda x: x[1])[1:6]

[(721, 0.4491629610875232),
 (10, 0.4421256757101673),
 (437, 0.38883963059318133),
 (3068, 0.3787610270497935),
 (11328, 0.3742405942825599)]

In [307]:
# Criando função de recomendação

def recommend (movie):
  # Número do filme
  anime_index = new_animes[new_animes['title'] == movie].index[0]
  # Relacionar animes que você selecionou
  distances = similarity[anime_index]
  # Retorna 06 animes que relaciona com o que você quer
  anime_list = sorted(list(enumerate(distances)), reverse= True, key= lambda x: x[1])[1:6]

  for i in anime_list:
    print(new_animes.iloc[i[0]].title)

In [341]:
animes['title'][80]

'Mobile Suit Gundam Wing: Endless Waltz Movie'

In [343]:
recommend("Neko mo, Onda-ke")

Tokyo Juushouden
Devilman: Yochou Sirene-hen
Nanatsu no Taizai: Imashime no Fukkatsu
Legend of Basara
Dragon Slayer Eiyuu Densetsu: Ouji no Tabidachi
